In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

In [3]:
data = pd.read_csv('no_hashtag.csv')

In [4]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [ ]:
from gensim.models import Word2Vec
from nltk import word_tokenize

X_train_token = X_train.map(word_tokenize)
model = Word2Vec(X_train_token, window=5, min_count=1, workers=4)

In [ ]:
from sklearn.decomposition import PCA

vectors = model[model.wv.vocab]
words = list(model.wv.vocab)
pca = PCA(n_components=2)
PCA_result = pca.fit_transform(vectors)

words = pd.DataFrame(words)
PCA_result = pd.DataFrame(PCA_result)
PCA_result['x_values'] =PCA_result.iloc[0:, 0]
PCA_result['y_values'] =PCA_result.iloc[0:, 1]
PCA_final = pd.merge(words, PCA_result, left_index=True, right_index=True)
PCA_final['word'] =PCA_final.iloc[0:, 0]
PCA_data_complet =PCA_final[['word','x_values','y_values']]

In [ ]:
X_train_pca = []
for i in range(len(PCA_data_complet)):
    a = PCA_data_complet.loc[i, 'x_values']
    b = PCA_data_complet.loc[i, 'y_values']
    X_train_pca.append([a,b])

In [ ]:
X_train_cluster = np.array(X_train_pca)

In [ ]:
from sklearn.cluster import KMeans
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters,
                random_state=0, 
                n_init=10,
                init='k-means++',
                verbose=0, 
                max_iter=300).fit(X_train_cluster)

print("clusters: ", num_clusters, "\tcluster inertia: ", kmeans.inertia_)

print("cluster centers:\n", kmeans.cluster_centers_)
# open('cluster_centres.csv', 'w').writelines(','.join(map(str, kmeans.cluster_centers_.tolist())))

cluster_labels = kmeans.labels_

In [ ]:
corpus = ' '.join(sorted(set([text for text in data])))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def get_vectors(*strs):
    text = [t for t in strs]
    vectorizer = TfidfVectorizer(text)
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()

data_vector = get_vectors(corpus)

In [ ]:
vectorizer = TfidfVectorizer()

data_vectorizer = vectorizer.fit_transform(data)